In [10]:
import pandas as pd
import Levenshtein

In [11]:
words = pd.read_csv('words.csv')

In [12]:
words.head()

,Français,Hiragana,Kanji
0,enseignant,きょうし,教室
1,étudiant,がくせい,学生
2,employé d'une compagnie,かいしゃいん,会社員
3,employé (avec nom de compagnie),しゃいん,社員
4,employé de banque,ぎんこういん,銀行員


In [13]:
shuffled_words = words.sample(frac=1).reset_index(drop=True)

In [14]:
def is_similar_japanese_answer(user_answer, correct_answer):
    similarity_threshold = 0.8
    distance = Levenshtein.distance(user_answer.lower(), correct_answer.lower())
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    return similarity >= similarity_threshold

In [15]:
def is_similar_french_answer(user_answer, correct_answer):
    similarity_threshold = 0.7
    if pd.isna(user_answer) or pd.isna(correct_answer):
        return False
    user_answer = user_answer.lower()
    correct_answer = correct_answer.lower()
    substring_match = user_answer in correct_answer or correct_answer in user_answer
    if substring_match:
        return True
    distance = Levenshtein.distance(user_answer, correct_answer)
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    
    return similarity >= similarity_threshold

In [16]:
shuffled_sample = shuffled_words.sample(n=20)
shuffled_sample

,Français,Hiragana,Kanji
137,université,だいがく,大学
118,étagère,たな,棚
21,petite monnaie,こまかいおかね,細かいお金
127,condition,ちょうし,調子
6,employé (avec nom de compagnie),しゃいん,社員
142,horaire,じこくひょう,時刻表
19,boîte,はこ,箱
144,étudiant étranger,りゅうがくせい,留学生
116,spécial express,とっきゅう,特急
25,cette année,ことし,NaN


In [17]:
seen_fr = []
correct_count = 0
score_fr = 0
shuffled_sample_jp = shuffled_sample.sample(frac=1).reset_index(drop=True)

for i in range(len(shuffled_sample_jp)):
    if shuffled_sample_jp['Français'].iloc[i] not in seen_fr:
        ask_h = input(f"Comment dire {shuffled_sample_jp['Français'].iloc[i]} en hiragana ?")
        seen_fr.append(shuffled_sample_jp['Français'].iloc[i])

        if is_similar_japanese_answer(ask_h, shuffled_sample_jp['Hiragana'].iloc[i]):
            kanji_value = shuffled_sample_jp['Kanji'].iloc[i]
            if pd.notna(kanji_value):
                ask_k = input("Et en kanji ?")
                if is_similar_japanese_answer(ask_k, kanji_value):
                    correct_count += 1
                    score_fr += 1
                    print('Correct !')
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
                else:
                    print(f"Incorrect ! La bonne réponse est : {kanji_value}")
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
            else:
                correct_count += 1
                score_fr += 1
                print('Correct !')
                print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
        else:
            print(f"Incorrect ! La bonne réponse est : {shuffled_sample_jp['Hiragana'].iloc[i]}")
            print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')

if correct_count == len(shuffled_sample_jp):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")
else:
    print(f"Vous avez répondu correctement à {score_fr} des {len(shuffled_sample_jp)} questions.")

Comment dire (ma) femme en hiragana ?つま
Et en kanji ?妻
Correct !
Bonnes réponses : 1/20
Comment dire cette année en hiragana ?ことし
Correct !
Bonnes réponses : 2/20
Comment dire petite monnaie en hiragana ?こまかいおかね
Et en kanji ?細かいお金
Correct !
Bonnes réponses : 3/20
Comment dire accueillir / aller chercher en hiragana ?むかえります
Et en kanji ?迎えります
Correct !
Bonnes réponses : 4/20
Comment dire horaire en hiragana ?じかん
Incorrect ! La bonne réponse est : じこくひょう
Bonnes réponses : 4/20
Comment dire condition en hiragana ?ちょうし
Et en kanji ?調子
Correct !
Bonnes réponses : 5/20
Comment dire express en hiragana ?きゅうこう
Et en kanji ?急行
Correct !
Bonnes réponses : 6/20
Comment dire étagère en hiragana ?たな
Et en kanji ?棚
Correct !
Bonnes réponses : 7/20
Comment dire montant/somme d'argent en hiragana ?
Incorrect ! La bonne réponse est : きんがく
Bonnes réponses : 7/20
Comment dire devoirs en hiragana ?しゅうくだい
Et en kanji ?宗育大
Incorrect ! La bonne réponse est : 宿題
Bonnes réponses : 7/20
Comment dire hobby en hi

In [ ]:
seen_k=[]
correct_count = 0
score_jp = 0
shuffled_sample_fr = shuffled_sample.sample(frac=1).reset_index(drop=True)
for i in range(len(shuffled_sample_fr)):
    kanji = shuffled_sample_fr['Kanji'].iloc[i]
    if pd.notna(kanji) and kanji not in seen_k:
        ask_h = input(f"Que veut dire ce kanji : {kanji}?")
        if is_similar_french_answer(ask_h, shuffled_sample_fr['Français'].iloc[i]):
            seen_k.append(kanji)
            correct_count += 1
            score_jp += 1
            print(f"Correct ! C'était bien {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr)}')
        else:
            print(f"Incorrecte ! La bonne réponse était : {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr)}')

if correct_count == len(shuffled_sample_fr):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")
else:
    print(f"Vous avez répondu correctement à {score_jp} des {len(shuffled_sample_fr)} questions.")

Que veut dire ce kanji : 細かいお金?petite monnaie
Correct ! C'était bien petite monnaie
Bonnes réponses : 1/20
Que veut dire ce kanji : 趣味?hobby
Correct ! C'était bien hobby
Bonnes réponses : 2/20
Que veut dire ce kanji : 特急?spécial express
Correct ! C'était bien spécial express
Bonnes réponses : 3/20
Que veut dire ce kanji : 宿題?devoirs
Correct ! C'était bien devoirs
Bonnes réponses : 4/20
Que veut dire ce kanji : 迎えります?accueillir
Correct ! C'était bien accueillir / aller chercher
Bonnes réponses : 5/20
Que veut dire ce kanji : 社員?employé (d'une compagnie
Incorrecte ! La bonne réponse était : employé (avec nom de compagnie)
Bonnes réponses : 5/20
Que veut dire ce kanji : 留学生?étudiant
Correct ! C'était bien étudiant étranger
Bonnes réponses : 6/20
Que veut dire ce kanji : 休みます?se reposer
Correct ! C'était bien se reposer / prendre un congé
Bonnes réponses : 7/20
Que veut dire ce kanji : 箱?boite
Correct ! C'était bien boîte
Bonnes réponses : 8/20
